In [13]:
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import os

# CSV保存先
filename = "hottomotto_products.csv"

# 栄養成分パース関数
def parse_nutrition(soup):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A", 
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }
    
    # 普通盛り（is_showクラス）のタブから栄養成分を取得
    nutrition_selectors = [
        "li.c-tab__cont-item.is_show div.c-table-type01 dl.c-table__item",
        "li.c-tab__cont-item.js-tab__cont.is_show div.c-table-type01 dl.c-table__item",
        "div.c-table-type01 dl.c-table__item",
        "dl.c-table__item"
    ]
    
    dls = []
    for selector in nutrition_selectors:
        dls = soup.select(selector)
        if dls:
            print(f"栄養成分を {len(dls)} 件発見 (セレクタ: {selector})")
            break
    
    if not dls:
        print("栄養成分が見つかりませんでした")
        return nutrition
    
    for dl in dls:
        dt_elem = dl.select_one("dt.c-table__head, dt")
        dd_elem = dl.select_one("dd.c-table__body, dd")
        
        if dt_elem and dd_elem:
            # dtからテキストを取得（spanタグ内の単位情報は除外）
            dt_text = dt_elem.get_text(strip=True)
            dd_text = dd_elem.get_text(strip=True)
            
            print(f"栄養成分項目: '{dt_text}' = '{dd_text}'")
            
            # 数値部分を抽出
            dd_clean = dd_text.strip()
            
            if "熱量" in dt_text:
                nutrition["calorie"] = dd_clean + "kcal"
            elif ("蛋白質" in dt_text or "蛋⽩質" in dt_text or 
                  "たんぱく質" in dt_text or "タンパク質" in dt_text):
                nutrition["protein"] = dd_clean + "g"
            elif "脂質" in dt_text:
                nutrition["fat"] = dd_clean + "g"
            elif ("炭水化物" in dt_text or "炭⽔化物" in dt_text):
                nutrition["carbohydrate"] = dd_clean + "g"
            elif "糖質" in dt_text:
                nutrition["sugar"] = dd_clean + "g"
            elif "食物繊維" in dt_text:
                nutrition["fiber"] = dd_clean + "g"
            elif ("食塩相当量" in dt_text or "⾷塩相当量" in dt_text or "塩分" in dt_text):
                nutrition["salt"] = dd_clean + "g"
    
    return nutrition

# 商品ページ取得
def scrape_hottomotto_product(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.text, "html.parser")
        
        # 商品名
        product_name = "N/A"
        title_selectors = [
            "h1.c-menu__title",
            "h1",
            ".c-menu__title", 
            ".product-title",
            "h2"
        ]
        
        for selector in title_selectors:
            title_tag = soup.select_one(selector)
            if title_tag:
                product_name = title_tag.get_text(strip=True)
                break
        
        # 価格
        price = "N/A"
        price_selectors = [
            ".c-menu__price",
            ".price",
            ".cost",
            "[class*='price']"
        ]
        
        for selector in price_selectors:
            price_tag = soup.select_one(selector)
            if price_tag:
                price_text = price_tag.get_text(strip=True)
                # 価格から余計な文字を除去し、円を付加
                price_clean = ''.join(filter(str.isdigit, price_text))
                if price_clean:
                    price = price_clean + "円"
                break
        
        # 画像URL
        picture_url = "N/A"
        img_selectors = [
            ".c-menu__pic img",
            ".product-image img",
            "img[src*='menu']",
            "img[src*='product']"
        ]
        
        for selector in img_selectors:
            img_tag = soup.select_one(selector)
            if img_tag and img_tag.get("src"):
                src = img_tag["src"]
                if src.startswith("http"):
                    picture_url = src
                else:
                    picture_url = urljoin("https://www.hottomotto.com", src)
                break
        
        # 栄養成分
        nutrients = parse_nutrition(soup)
        
        return {
            "product_name": product_name,
            "price": price, 
            "url": url,
            **nutrients,
        }
        
    except Exception as e:
        print(f"商品ページ取得エラー {url}: {e}")
        return None

# 一覧ページから商品URL取得
def get_product_urls(list_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(list_url, headers=headers)
        response.encoding = response.apparent_encoding  
        soup = BeautifulSoup(response.text, "html.parser")
        
        urls = set()
        
        # 画像の構造に基づいて、商品へのリンクを検索
        # 複数のパターンを試す
        link_patterns = [
            "a[href*='/products/']",
            "a[href*='/menu/']", 
            "a[href*='/item/']",
            ".c-menu__item a",
            ".product-item a",
            "li a[href]"
        ]
        
        for pattern in link_patterns:
            links = soup.select(pattern)
            for link in links:
                href = link.get("href")
                if href and ("/products/" in href or "/menu/" in href or "/item/" in href):
                    full_url = urljoin("https://www.hottomotto.com", href)
                    urls.add(full_url)
        
        # デバッグ：リンクが見つからない場合
        if not urls:
            print("商品リンクが見つかりません。すべてのリンクを確認中...")
            all_links = soup.find_all('a', href=True)
            print(f"ページ内の全リンク数: {len(all_links)}")
            
            # 商品らしきリンクを探す
            for link in all_links:
                href = link.get('href', '')
                text = link.get_text(strip=True)
                if any(keyword in href.lower() for keyword in ['product', 'menu', 'item', 'donut']):
                    full_url = urljoin("https://www.hottomotto.com", href)
                    urls.add(full_url)
                    print(f"  見つかったリンク: {text} -> {full_url}")
        
        return list(urls)
        
    except Exception as e:
        print(f"一覧ページ取得エラー {list_url}: {e}")
        return []

# CSV保存
def save_to_csv(data, filename=filename):
    if not data:
        print("保存するデータがありません")
        return
        
    fieldnames = [
        "product_name", "price", "url",
        "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"
    ]
    
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

# メイン実行
if __name__ == "__main__":
    # 商品一覧ページURL（画像で見えているページ）
    list_url = "https://www.hottomotto.com/menu_list/index/13"
    
    print(f"一覧ページを取得中: {list_url}")
    product_urls = get_product_urls(list_url)
    print(f"商品URL取得数: {len(product_urls)}")
    
    if not product_urls:
        print("商品URLが見つかりませんでした。URLやページ構造を確認してください。")
        exit(1)
    
    # 最初の数件のURLを表示
    print("\n取得した商品URL（最初の5件）:")
    for i, url in enumerate(product_urls[:5]):
        print(f"  {i+1}. {url}")
    
    # 各商品ページのデータを取得
    all_data = []
    failed_urls = []
    
    for i, url in enumerate(product_urls, 1):
        print(f"\n[{i}/{len(product_urls)}] 処理中: {url}")
        
        try:
            data = scrape_hottomotto_product(url)
            if data:
                all_data.append(data)
                print(f"✓ 成功: {data['product_name']} - {data['price']}")
            else:
                failed_urls.append(url)
                print(f"✗ データ取得失敗")
            
            # サーバー負荷軽減
            time.sleep(1)
            
        except Exception as e:
            failed_urls.append(url)
            print(f"✗ エラー: {e}")
    
    # 結果まとめ
    print(f"\n=== 結果 ===")
    print(f"成功: {len(all_data)} 件")
    print(f"失敗: {len(failed_urls)} 件")
    
    if failed_urls:
        print("\n失敗したURL:")
        for url in failed_urls[:5]:  # 最初の5件のみ表示
            print(f"  - {url}")
    
    # CSV保存
    if all_data:
        save_to_csv(all_data)
        print(f"\nCSV保存完了: {filename} ({len(all_data)}件)")
        
        # サンプルデータ表示
        print(f"\n=== サンプルデータ ===")
        for data in all_data[:3]:  # 最初の3件のみ表示
            print(f"商品名: {data['product_name']}")
            print(f"価格: {data['price']}")
            print(f"カロリー: {data['calorie']}")
            print(f"蛋白質: {data['protein']}")
            print(f"脂質: {data['fat']}")
            print(f"炭水化物: {data['carbohydrate']}")
            print(f"糖質: {data['sugar']}")
            print(f"食物繊維: {data['fiber']}")
            print(f"食塩相当量: {data['salt']}")
            print(f"URL: {data['url']}")
            print("-" * 50)
    else:
        print("保存するデータがありませんでした。")

一覧ページを取得中: https://www.hottomotto.com/menu_list/index/13
商品リンクが見つかりません。すべてのリンクを確認中...
ページ内の全リンク数: 150
  見つかったリンク: 印刷用PDFのダウンロード -> https://www.hottomotto.com/menu_list/download_pamphlet/13/09kantouA_toukai_kansaiA.pdf?id=858
  見つかったリンク: アレルギー -> https://www.hottomotto.com/menu_list/info/13
  見つかったリンク: 栄養成分 -> https://www.hottomotto.com/menu_list/info/13#!/tab/eiyo
  見つかったリンク: 原産国 -> https://www.hottomotto.com/menu_list/info/13#!/tab/origin
  見つかったリンク: ふわっと香る炭火旨だれ焼鳥つくね重660円（税抜：612円） -> https://www.hottomotto.com/menu_list/view/13/1287
  見つかったリンク: ふわっと香る炭火W旨だれ焼鳥つくね重(肉2倍)1,050円（税抜：973円） -> https://www.hottomotto.com/menu_list/view/13/1288
  見つかったリンク: とろ～り絡まる温玉付き焼鳥つくね重730円（税抜：676円） -> https://www.hottomotto.com/menu_list/view/13/1289
  見つかったリンク: とろ～り絡まる温玉付きW焼鳥つくね重(肉2倍)1,120円（税抜：1,038円） -> https://www.hottomotto.com/menu_list/view/13/1290
  見つかったリンク: 野菜が摂れる！特製ナポリタン＆サラダ500円（税抜：463円） -> https://www.hottomotto.com/menu_list/view/13/1291
  見つかったリンク: 野菜が摂れる！W特製ナポリタン＆サラダ(麺2倍)650円（税抜：602円） -> http